# Ensemble Methods Applied

Agenda:
- Review code for Voting Classifier, Bagging Classifier, and Random Forest
- Practice finding optimal hyperparameter for  Random Forest with gridsearch


## Import and Prep Titanic dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier, RandomForestRegressor, ExtraTreesRegressor


In [2]:
# Read in data and split data to be used in the models
titanic = pd.read_csv('https://raw.githubusercontent.com/learn-co-students/nyc-ds-033020-lectures/master/Mod_3/decision_trees/cleaned_titanic.csv', index_col='PassengerId')



In [3]:
# Create matrix of features
X = titanic.drop('Survived', axis = 1) # grabs everything else but 'Survived'

# Create target variable
y = titanic['Survived'] # y is the column we're trying to predict

# Create a list of the features being used in the 
feature_cols = X.columns

In [4]:
# Use x and y variables to split the training data into train and test set then scale that data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

## Fit a KNN model

In [5]:
from sklearn.neighbors import KNeighborsClassifier

In [6]:
knn = KNeighborsClassifier(n_neighbors=8)

In [7]:
knn.fit(X_train, y_train)

knn_preds = knn.predict(X_test)

knn_f1 = metrics.f1_score(y_test, knn_preds)


print(knn_f1)

0.7770700636942675


## Fit a Logistic Regression model 

In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
lr = LogisticRegression(class_weight='balanced')

In [10]:
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
lr_preds = lr.predict(X_test)

lr_f1 = metrics.f1_score(y_test, lr_preds)

print(lr_f1)

0.8066298342541436


## Fit a Decision Tree Classifier

In [12]:
from sklearn.tree import DecisionTreeClassifier

In [13]:
dtc = DecisionTreeClassifier(max_depth=_____, class_weight='balanced')

dtc.fit(X_train, y_train)

dtc_preds  = dtc.predict(X_test)

dtc_f1 = metrics.f1_score(y_test, dtc_preds)

print(dtc_f1)

0.8047337278106509


## Combine three models using Voting Classifier

In [14]:
from sklearn.ensemble import VotingClassifier


For the estimators, we must provide a list of tuples. The first value in the tuple is is the name given to the model/estimator in the second value. SKlearn requires this because there is additional functionality where you can access information about the specific models, so you need to name the models to access them later.  

In [15]:
voting_clf = VotingClassifier(
                estimators=[('logreg', ____), ('knneighbors', ____), ('decisiontree', ____)], 
                voting=____)

voting_clf.fit(X_train, y_train)

vc_preds = voting_clf.predict(____)

vc_f1 = metrics.f1_score(____, ____)

print(vc_f1)

0.8255813953488372


### Use a voting classifier with multiple Logistic regression models 

In [20]:

C_param_range = [0.001,0.01,0.1,1,10]
titles = ['lr_0_001', 'lr_0_01', 'lr_0_1', 'lr_1', 'lr_10']

params = dict(zip(titles, C_param_range)) 
models = {}

table = pd.DataFrame(columns = ['C_parameter','F1'])
table['C_parameter'] = C_param_range
j = 0

for k , v  in params.items():
    
    # Create model using different value for c  
    lr = LogisticRegression(penalty = 'l2', C = v, random_state = 1, class_weight='balanced')
    
    #save the model to a dictionary to use later in our voting classifiers
    models[k]= lr
    
    #the steps below this point are unnecessary in order to create a voting classifier, 
    #but it is easy to fit the model and see how performance changes for different levels of regularization
    lr.fit(X_train, y_train)
    
    # Predict using model
    y_preds = lr.predict(X_test)

    # Saving accuracy score in table
    table.iloc[j,1] = metrics.f1_score(y_test, y_preds)
    j += 1



starting


In [21]:
#review performance for different levels of C
table


,C_parameter,F1
0,0.001,0.735135
1,0.010,0.751381
2,0.100,0.804469
3,1.000,0.80663
4,10.000,0.80663


In [22]:
#invesitgate the models D=dictionary
list(models.items())

[('lr_0_001',
  LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                     fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                     max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                     random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                     warm_start=False)),
 ('lr_0_01',
  LogisticRegression(C=0.01, class_weight='balanced', dual=False,
                     fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                     max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                     random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                     warm_start=False)),
 ('lr_0_1',
  LogisticRegression(C=0.1, class_weight='balanced', dual=False,
                     fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                     max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                     random_state=1, solver='lbfg

Now that we have programmatically created multiple logistic regression models, let's use them in an ensemble model

In [23]:
lr_voting = VotingClassifier(estimators=list(models.items()), 
                              voting='hard')

lr_voting.fit(X_train, y_train)

lrv_preds = voting_clf.predict(X_test)

lrv_f1 = metrics.f1_score(y_test, lrv_preds)

print(vc_f1)

0.8255813953488372


## Fit a Bagging Classifier for a Logistic Regression model. 

In [28]:
bc_lr = BaggingClassifier(
            base_estimator=LogisticRegression(random_state = 1, class_weight='balanced'), 
            n_estimators= ____,
            max_samples= ____,
            max_features= ____,
            oob_score= True
                )

In [29]:
bc_lr.fit(X_train, y_train)



BaggingClassifier(base_estimator=LogisticRegression(C=1.0,
                                                    class_weight='balanced',
                                                    dual=False,
                                                    fit_intercept=True,
                                                    intercept_scaling=1,
                                                    l1_ratio=None, max_iter=100,
                                                    multi_class='auto',
                                                    n_jobs=None, penalty='l2',
                                                    random_state=1,
                                                    solver='lbfgs', tol=0.0001,
                                                    verbose=0,
                                                    warm_start=False),
                  bootstrap=True, bootstrap_features=False, max_features=6,
                  max_samples=0.7, n_estimators=100, n_jobs=None,

In [30]:
# Use the oob_score to get some idea of how the model performs on a validation set

bc_lr.oob_score_

0.7762762762762763

In [31]:
# See how the model performs on the test set

bc_lr_preds = bc_lr.predict(X_test)

bc_lr_f1 = metrics.f1_score(y_test, bc_lr_preds)

print(bc_lr_f1)

0.7826086956521741


***What is the difference in the `VotingClassifier` algorithm and the `BaggingClassifier` algorithm?***

Your answer:

# Fitting a Random Forest Classifier

In [36]:
# Instantiate the classifier using 100 trees
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state = 1, n_estimators=100, max_depth=1, max_features=4)

In [37]:
#let's look at all the different default features
rfc

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=1, max_features=4,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [38]:
#fit the model to the training data
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=1, max_features=4,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [39]:
#use the fitted model to predict on the test data
rfc_preds = rfc.predict(X_test)

rfc_f1 = metrics.f1_score(y_test, rfc_preds)

# checking accuracy on the test data
print('Test F1 score: ', rfc_f1)

Test F1 score:  0.7074829931972789


***Increase the number of trees and see how the model performs***

### GridsearchCV with Random Forest

Let's use grid search to identify the best tuning parameters to use for a random forest model. 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#create a dictionary of all the parameters you want to tune
param_grid = { 
    param1: list_of_options,
    param2: list_of_options,
    param3: list_of_options,
    param4: list_of_options,
}

In [ ]:
#create a grid search object and fit it to the data



In [ ]:
### Identify the best params 



#Identify the best score during fitting with cross-validation


In [ ]:
#predict on the test set


# checking accuracy


# checking accuracy
